In [1]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display


# object detection module
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# import config
import config

# PATCHES
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

def load_model(path):
    model = tf.saved_model.load(path)
    model = model.signatures['serving_default']
    return model


def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis,...]

    # Run inference
    output_dict = model(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections

    # detection_classes should be ints.
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)

    # Handle models with masks:
    if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
                  output_dict['detection_masks'], output_dict['detection_boxes'],
                   image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                           tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()

    return output_dict


def show_inference(model, image_path, category_index):
    
    # the array based representation of the image will be used later in order to prepare the
    # result image with boxes and labels on it.
    image_np = np.array(Image.open(image_path))
    # Actual detection.
    output_dict = run_inference_for_single_image(model, image_np)
    # Visualization of the results of a detection.
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)    

    return image_np, output_dict






class object_detector:
    def __init__(self):
        self.path_to_model = config.PATH_MODEL
        self.path_to_label = config.PATH_TO_LABELS
        self.model = load_model(self.path_to_model)
        self.category_index = label_map_util.create_category_index_from_labelmap(config.PATH_TO_LABELS, use_display_name=True)
        
    def inference(self,image_path, save_image=True):
        image_np, output = show_inference(self.model, image_path, self.category_index)
        if save_image:
            im = Image.fromarray(image_np)
            im.save("saved_image.jpeg")            
        return {'scores':output['detection_scores'],'classes':output['detection_classes'],'status':'OK'}
    
    
    
    

    



In [2]:
od = object_detector()

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [3]:
od.inference(config.PATH_IMAGE)

{'scores': array([0.94069064, 0.93450266, 0.2308823 , 0.2251894 , 0.17249915,
        0.1396198 , 0.13212033, 0.09975749, 0.08992708, 0.08888808,
        0.08091756, 0.07895234, 0.07818329, 0.07565755, 0.07331286,
        0.07074891, 0.06597457, 0.06404546, 0.06387997, 0.0638117 ,
        0.06380183, 0.06050139, 0.05898515, 0.058983  , 0.0576143 ,
        0.05641402, 0.05638803, 0.0550001 , 0.0549573 , 0.05426418,
        0.05425417, 0.05409578, 0.05335932, 0.05279239, 0.05228513,
        0.05219475, 0.05164074, 0.05066017, 0.05030197, 0.04837742,
        0.04822731, 0.04720661, 0.04673678, 0.0467178 , 0.04644242,
        0.04614913, 0.04420191, 0.04286546, 0.04251259, 0.04174345,
        0.04166575, 0.04137343, 0.04096131, 0.0405353 , 0.03990964,
        0.03988717, 0.03984373, 0.0393099 , 0.03923892, 0.03877983,
        0.03877517, 0.03853002, 0.03816222, 0.03815905, 0.03742462,
        0.03689887, 0.03677595, 0.03671168, 0.03654769, 0.03635332,
        0.03612375, 0.03610821, 0.0360